![book](https://raw.githubusercontent.com/ageron/tensorflow-safari-course/master/images/intro_to_tf_course.png)

**Try not to peek at the solutions when you go through the exercises. ;-)**

First let's make sure this notebook works well in both Python 2 and Python 3:

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf
tf.__version__

/home/farid/.virtualenvs/tensorflow-safari-course/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


'1.4.1'

## Variables

In [ ]:
>>> graph = tf.Graph()
>>> with graph.as_default():
...     x = tf.Variable(100)
...     c = tf.constant(5)
...     increment_op = tf.assign(x, x + c)
...

In [ ]:
>>> with tf.Session(graph=graph) as sess:
...     x.initializer.run()
...     print(x.eval())    # 100
...     for iteration in range(10):
...         increment_op.eval()
...     print(x.eval())    # 150
...

## Variables Initializer

In [ ]:
>>> graph = tf.Graph()
>>> with graph.as_default():
...     x = tf.Variable(100)
...     c = tf.constant(5)
...     increment_op = tf.assign(x, x + c)
...     init = tf.global_variables_initializer()
...

In [ ]:
>>> with tf.Session(graph=graph) as sess:
...     init.run()
...     print(x.eval())    # 100
...     for iteration in range(10):
...         increment_op.eval()
...     print(x.eval())    # 150
...

## Variable State

In [ ]:
>>> session1 = tf.Session(graph=graph)
>>> session2 = tf.Session(graph=graph)
>>> x.initializer.run(session=session1)
>>> x.initializer.run(session=session2)

In [ ]:
>>> increment_op.eval(session=session1)

In [ ]:
>>> x.eval(session=session1)

In [ ]:
>>> x.eval(session=session2)

In [ ]:
>>> session1.close()
>>> session2.close()

## Exercise 2

![Exercise](https://c1.staticflickr.com/9/8101/8553474140_c50cf08708_b.jpg)

In [8]:
import numpy as np
from IPython.display import display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def=None, max_const_size=32):
    """Visualize TensorFlow graph."""
    graph_def = graph_def or tf.get_default_graph()
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In this exercise, we will use TensorFlow to compute $ 1 + \frac{1}{2} + \frac{1}{4} + \frac{1}{8} + \cdots $ by creating a simple graph then running it multiple times.

Think about how you would solve this problem (and if you are feeling confident enough, go ahead and implement your ideas), then follow the instructions below.

In [15]:
graph = tf.Graph()

with graph.as_default():
    y = tf.Variable(0.,name='y')
    x = tf.Variable(1.,name='x')
    increment_op = tf.assign(y, y + x)
    divide_op = tf.assign(x, x/2)
    init = tf.global_variables_initializer()



In [13]:
with tf.Session(graph=graph) as sess:
    init.run()
    print(sess.run([x,y]))    
    for iteration in range(100):
        increment_op.eval()
        divide_op.eval()
    print(sess.run([x,y]))    

[1.0, 0.0]
[7.8886091e-31, 2.0]


In [45]:
x,y, x.op.inputs, x.op.outputs, y.op.inputs, y.op.outputs, y.op.

(<tf.Variable 'Variable:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=() dtype=float32_ref>,
 [<tf.Tensor 'Variable:0' shape=() dtype=float32_ref>],
 [<tf.Tensor 'Variable_1:0' shape=() dtype=float32_ref>],
 <tf.Operation 'Variable_1' type=VariableV2>)

In [14]:
show_graph(graph)

2.1) Create a graph with two variables $ x $ and $ y $, initialized to 0.0 and 1.0 respectively. Create an operation that will perform the following assignment: $ x \gets x + y $. Create a second operation that will perform the following assignment: $ y \gets \dfrac{y}{2} $.

2.2) Now create a `Session()`, initialize the variables, then create a loop that will run 50 times, and at each iteration will run the first assignment operation, then the second (separately). Finally, print out the value of $ x $. The result should be very close (or equal to) 2.0.

2.3) Try to run the assignment operations simultaneously. What happens to the result? Run your code multiply times: do the results vary? Can you explain what is happening?

2.5) Bonus question (if you have time): update you graph to define the second assignment ($y \gets \frac{y}{2}$) inside a `tf.control_dependencies()` block, to  guarantee that it runs after the first assignment ($ x \gets x + y$). Does this finally solve the problem?

Try not to peek at the solution below before you have done the exercise! :)

![thinking](https://upload.wikimedia.org/wikipedia/commons/0/06/Filos_segundo_logo_%28flipped%29.jpg)

## Exercise 2 - Solution

2.1)

In [32]:
graph = tf.Graph()
with graph.as_default():
    x = tf.Variable(0.0)
    y = tf.Variable(1.0)
    add = tf.assign(x, x + y)
    divide = tf.assign(y, y / 2)
    init = tf.global_variables_initializer()

2.2)

In [33]:
with tf.Session(graph=graph):
    init.run()
    for iteration in range(20):
        add.eval()
        divide.eval()
    result = x.eval()

In [34]:
print(result)

2.0


2.3)

In [37]:
with tf.Session(graph=graph) as sess:
    init.run()
    for iteration in range(20):
        sess.run([add, divide])
    result = x.eval()

In [38]:
result

1.7393761

2.4)

In [21]:
graph = tf.Graph()
with graph.as_default():
    x = tf.Variable(0.0)
    y = tf.Variable(1.0)
    add = tf.assign(x, x + y)
    with tf.control_dependencies([add]):
        divide = tf.assign(y, y / 2)
    init = tf.global_variables_initializer()

In [22]:
with tf.Session(graph=graph) as sess:
    init.run()
    for iteration in range(30):
        sess.run([add, divide])
    result = x.eval()

In [23]:
result

2.0